In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    'Resources/crypto_market_data.csv',
    index_col='coin_id')

# Display sample data
df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(
    data=StandardScaler().fit_transform(df),
    # Copy the crypto names from the original data
    columns=df.columns,
    # Set the coinid column as index
    index=df.index
)
# Display sample data
df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [6]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {
    # Create a list with the number of k-values from 1 to 11
    'k': list(range(1, 12)),
    # Create an empty list to store the inertia values
    'inertia': []
}
# Create a for loop to compute the inertia with each possible value of k
for i in elbow_dict['k']:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=i, random_state=1, n_init=10)
    # 2. Fit the model to the scaled data
    model.fit(df_scaled)
    # 3. Append the model.inertia_ to the inertia list
    elbow_dict['inertia'].append(model.inertia_)
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_dict)
elbow_df.head()

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,63.858668


In [7]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x='k', 
    y='inertia', 
    title='Elbow Curve',
    xticks=elbow_dict['k']
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** From the plot, I would guess 4, since that is the point where the slope stabilizes and starts dropping more steadily.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [8]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1, n_init=10)

In [9]:
# Fit the K-Means model using the scaled data
model.fit(df_scaled)

KMeans(n_clusters=4, n_init=10, random_state=1)

In [10]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predicted_cluster = model.predict(df_scaled)

# Print the resulting array of cluster values.
predicted_cluster

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [11]:
# Create a copy of the DataFrame
df_predictions = df.copy()

In [12]:
# Add a new column to the DataFrame with the predicted clusters
df_predictions['cluster'] = predicted_cluster
# reset index to set coin_id as a regular column and displya it in plot
df_predictions.reset_index(drop=False, inplace=True)
# Display sample data
df_predictions.head()

,coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster
0,bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,2
1,ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,2
2,tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
3,ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
4,bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,2


In [13]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_predictions.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [14]:
df_predictions.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_1y',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_1y,coin_id)

In [15]:
df_predictions.hvplot.scatter(
    x='price_change_percentage_7d',
    y='price_change_percentage_1y',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_7d]   (price_change_percentage_1y,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [20]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [21]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_scaled)
# View the first five rows of the DataFrame. 
pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [30]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
# Print the explained variance ratio fo each pc and total
print('Explained variance ratio')
print('  Each Principal Component')
[print(f'    {x:.1%}') for x in explained_variance]
print('  Total')
print(f'    {sum(explained_variance):.1%}')

Explained variance ratio
  Each Principal Component
    37.2%
    34.7%
    17.6%
  Total
    89.5%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total is 89.5%. Each principal component individually explains 37.2%, 34.7% and 17.6%.

In [31]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data, as index
pca_df.index = df.index

# Display sample data
pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [32]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {
    # Create a list with the number of k-values from 1 to 11
    'k': list(range(1, 12)),
    # Create an empty list to store the inertia values
    'inertia': []
}
# Create a for loop to compute the inertia with each possible value of k
for i in elbow_dict['k']:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=i, random_state=1, n_init=10)
    # 2. Fit the model to the pca data
    model.fit(pca_df)
    # 3. Append the model.inertia_ to the inertia list
    elbow_dict['inertia'].append(model.inertia_)
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_dict)
elbow_df.head()

,k,inertia
0,1,256.874086
1,2,165.901994
2,3,93.774626
3,4,49.665497
4,5,38.072792


In [33]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x='k', 
    y='inertia', 
    title='Elbow Curve',
    xticks=elbow_dict['k']
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** It seems to be 4, since that is where the elbow curve starts having a smooth slope.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, that is the same value I chose previously

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1, n_init=10)

In [ ]:
# Fit the K-Means model using the pca data
model.fit(pca_df)

KMeans(n_clusters=4, n_init=10, random_state=1)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the pca data
predicted_cluster = model.predict(pca_df)

# Print the resulting array of cluster values.
predicted_cluster

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [36]:
# Create a copy of the DataFrame
pca_df_predictions = pca_df.copy()

In [37]:
# Add a new column to the DataFrame with the predicted clusters
pca_df_predictions['cluster'] = predicted_cluster
# reset index to set coin_id as a regular column and displya it in plot
pca_df_predictions.reset_index(drop=False, inplace=True)
# Display sample data
pca_df_predictions.head()

,coin_id,PC1,PC2,PC3,cluster
0,bitcoin,-0.600667,0.842760,0.461595,2
1,ethereum,-0.458261,0.458466,0.952877,2
2,tether,-0.433070,-0.168126,-0.641752,0
3,ripple,-0.471835,-0.222660,-0.479053,0
4,bitcoin-cash,-1.157800,2.041209,1.859715,2


In [40]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_df_predictions.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC2,coin_id)

In [41]:
pca_df_predictions.hvplot.scatter(
    x='PC1',
    y='PC3',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC3,coin_id)

In [42]:
pca_df_predictions.hvplot.scatter(
    x='PC2',
    y='PC3',
    by='cluster',
    hover_cols='coin_id'
).opts(yformatter="%.0f")

:NdOverlay   [cluster]
   :Scatter   [PC2]   (PC3,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [30]:
# Composite plot to contrast the Elbow curves

In [31]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 